# AgroPulse Chatbot

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile")

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002537593B880>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025375939420>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env variables into environment

hf_token = os.getenv("HF_TOKEN")
os.environ["HF_TOKEN"] = hf_token


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\AgroPulse\ChatBot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [5]:
from bs4 import BeautifulSoup, SoupStrainer

In [6]:
import bs4


def load_web_docs(urls):
    all_docs = []
    for url in urls:
        loader = WebBaseLoader(
            web_paths=(url,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    class_=("post-content", "post-title", "post-header")
                )
            ),
        )
        docs = loader.load()
        all_docs.extend(docs)
    return all_docs

In [7]:
urls = [
    "https://invadeagro.com/2025/07/03/common-crop-diseases-india/",
    "https://eos.com/blog/crop-diseases/",
    "https://www.growag.com/highlights/article/guide-to-crop-disease-and-management",
    "https://kshema.co/blogs/common-crop-diseases-and-how-to-prevent-them/",
    "https://krikso.co.in/pest-and-disease-management-protecting-your-crops-effectively/",
    "https://blog.plantwise.org/tag/plant-diseases/",
    "https://farmonaut.com/blogs/best-farming-organic-farming-blogs-in-india",
    "https://agrisearchindia.com/en/blog"
]

docs = load_web_docs(urls)
print(f"Total docs: {len(docs)}")

Total docs: 8


In [8]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=Chroma.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002533305B4F0>, search_kwargs={'k': 3})

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## Prompt Template
system_prompt = (
    "You are an expert assistant for answering crop and farming questions. "
    "Use the following retrieved context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Always respond in the same language as the user's question. "
    "Provide a detailed, thorough, and informative answer, covering all relevant points. "
    "Explain technical terms if needed.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Please answer in detail {input}"),
    ]
)

In [10]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [11]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [12]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool
math_chain=LLMMathChain.from_llm(llm=llm)
calculator=Tool(
    name="Calculator",
    func=math_chain.run,
    description="A calculator for farming math. Use this when farmers ask about quantities, costs, yields, fertilizer requirements, or any arithmetic calculation."
)

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

In [14]:
# Wrap retriever as a tool
def run_qa_chain_tool(question):
    try:
        response = chain.invoke({"input": question})
        if isinstance(response, dict) and "result" in response:
            return response["result"]
        return response
    except Exception as e:
        return f"An error occurred: {e}"



retriever_tool = Tool(
    name="CropDocs",
    func=run_qa_chain_tool,
    description="Useful for answering farming and crop disease questions from AgroPulse blog database"
)

# Tools list (retriever + wiki + arxiv)
tools = [retriever_tool, wiki, calculator]

from langchain.agents import initialize_agent, Tool, AgentType
agent = initialize_agent(
    tools,
    llm,   
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


C:\Users\HP\AppData\Local\Temp\ipykernel_1684\3044567815.py:23: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [15]:
print(agent.run("What are the symptoms of rice blast disease?"))

C:\Users\HP\AppData\Local\Temp\ipykernel_1684\3098581047.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent.run("What are the symptoms of rice blast disease?"))




> Entering new AgentExecutor chain...
To answer this question, I need to find information about rice blast disease, which is a crop disease. The AgroPulse blog database, accessed through the CropDocs tool, is likely to have information about crop diseases, including rice blast disease.

Action: CropDocs
Action Input: rice blast disease symptoms
Observation: {'input': 'rice blast disease symptoms', 'context': [Document(id='7774f8d8-bf74-404b-8ecf-cc81f089b1e1', metadata={'source': 'https://kshema.co/blogs/common-crop-diseases-and-how-to-prevent-them/'}, page_content='Most common crop diseases caused by bacteria, fungi, nematodes, and virus need focused approach to contain the damage and save crops.\nBacterial diseases \nBlights and wilts are examples of bacterial common crop diseases. These affect plants when bacteria get into the culture’s tissue through damaged areas caused by agricultural tools, insects or unfavourable weather conditions. Infections may also occur when bacteria use

In [16]:
print(agent.run("And what is the best fertilizer for rice?"))



> Entering new AgentExecutor chain...
To answer this question, I need to find information about the best fertilizer for rice. Since this is a farming-related question, I should search for information in a database that specializes in farming and crop diseases.

Action: CropDocs
Action Input: best fertilizer for rice
Observation: {'input': 'best fertilizer for rice', 'context': [Document(id='5bba2605-e36e-451b-86a8-3f8c658416b6', metadata={'source': 'https://krikso.co.in/pest-and-disease-management-protecting-your-crops-effectively/'}, page_content='Crop Nutrition: Essential Fertilizers for Maximum Yield.Harvesting and Post-Harvest Management: Maximizing Your Yield.'), Document(id='64d2b76b-a991-4178-99d8-8dab86915ba2', metadata={'source': 'https://kshema.co/blogs/common-crop-diseases-and-how-to-prevent-them/'}, page_content='India has made tremendous strides in food-grain production from 82 million tonnes in 1960-61 to 271.37 million tonnes in 2018-19. It further increased to 329.7 m

In [17]:
print(agent.run("Remind me what we discussed about rice blast earlier."))



> Entering new AgentExecutor chain...
Thought: I don't have any record of our previous conversation about rice blast, so I need to find information about it from a reliable source.

Action: CropDocs
Action Input: rice blast
Observation: {'input': 'rice blast', 'context': [Document(id='743e38bf-0139-45ad-97e4-38e0be9fbb35', metadata={'source': 'https://kshema.co/blogs/common-crop-diseases-and-how-to-prevent-them/'}, page_content='Request Call Back\n\n\n\n\n\n\n\nName\r\n        *\n\n\n\n\nPhone Number\r\n        *\n\n\n\n\n\nEmail\r\n        *\n\n\n\n\nMessage\r\n        *\n\n\n\n\n\n\nSubmit\n\n\n\n\n\n\t\t\t\t\tIf you are human, leave this field blank.\t\t\t\t\n\n\nBy submitting this form, you agree to be contacted on your phone number or e-mail address by Kshema General Insurance and/or its representatives.\n\n\n\n\n\n\n\nRecent Posts\n\n\nPMFBY: The Story of Farmer Wellbeing in India\n\n\nSafe Harvesting Techniques to Protect Crops from Damage\n\n\nKharif Crop Yield: How to Estima